In [54]:
import pandas as pd
import numpy as np

attr_dict = {"age":0, "workclass":1, "fnlwgt":0, "education":1, "education-num":0, "marital-status":1, "occupation":1, "relationship":1, "race":1, "sex":1, "capital-gain":0, "capital-loss":0, "hours-per-week":0, "native-country":1, "salary":0} # 0: continuous, 1: discrete

train_data = pd.read_csv("adult.data",names=attr_dict.keys(),index_col=False)
# cut = int(0.9 * len(train_data))
cut = int(len(train_data))
train_data, validation_data = train_data[:cut], train_data[cut:]
test_data = pd.read_csv("adult.test",names=attr_dict.keys(),index_col=False,header=0)

# attributes = ["workclass","education","marital-status","occupation","relationship","race","sex","native-country","salary"]
# train_data = train_data[attributes] # only discrete values
# test_data = test_data[attributes] # only discrete values

attributes = list(attr_dict.keys())
attributes.remove("fnlwgt")
print(len(attributes))
train_data = train_data[attributes]
test_data = test_data[attributes]

# fill in ?
def fill_data(data):
    for a in attr_dict:
        if attr_dict[a]: # discrete
            data.loc[data[a] == " ?",a] = data[a].value_counts().argmax() # view or copy? Use loc!
        else: # continuous
            pass
            # print(a_col.value_counts())
            # a_col[a_col == " ?"] = np.mean(a_col[a_col != " ?"])

fill_data(train_data)
fill_data(test_data)

14


In [52]:
for a in attr_dict:
    print(a,len(train_data[a].unique()))

age 73
workclass 8
fnlwgt 21648
education 16
education-num 16
marital-status 7
occupation 14
relationship 6
race 5
sex 2
capital-gain 119
capital-loss 92
hours-per-week 94
native-country 41
salary 2


In [55]:
import random

def entropy(p):
    if p.ndim == 1:
        new_p = p[p != 0]
        return -np.sum(new_p * np.log2(new_p))
    else:
        # new_p = p[(p[:,0] != 0) & (p[:,1] != 0)]
        return -np.sum(p * np.log2(p),axis=1)

def information_gain(D,a,discrete_flag=False):
    pk = D["salary"].value_counts(normalize=True).values
    if discrete_flag:
        prop_Dv = D[a].value_counts(normalize=True).values # proportion
        prob_Dv = np.array([D.loc[D[a] == av]["salary"].value_counts(normalize=True).get(" >50K",0) for av in D[a].unique()])
        pp_stack = np.column_stack((prop_Dv,prob_Dv))
        pp_stack = pp_stack[(pp_stack[:,1] != 0) & (pp_stack[:,1] != 1)]
        prop_Dv = pp_stack[:,0]
        prob_Dv = pp_stack[:,1]
        prob_Dv_neg = 1 - prob_Dv
        return (entropy(pk) - np.sum(prop_Dv * entropy(np.column_stack((prob_Dv,prob_Dv_neg)))), a)
    else:
        a_sort = sorted(D[a].unique())
        # Ta = []
        # for i in range(len(a_sort) - 1):
        #     d1 = D[D[a] == a_sort[i]]["salary"].value_counts(normalize=True)
        #     d2 = D[D[a] == a_sort[i+1]]["salary"].value_counts(normalize=True)
        #     if not ((d1.get(" >50K",0) == 0 and d2.get(" >50K",0) == 0) or (d1.get(" <=50K",0) == 0 and d2.get(" <=50K",0) == 0)):
        #         Ta.append((a_sort[i] + a_sort[i+1]) / 2)
        Ta = [(a_sort[i] + a_sort[i+1]) / 2 for i in range(len(a_sort)-1)]
        # Ta = random.sample(Ta,min(len(Ta),100))
        # if len(Ta) == 0:
        #     Ta = [a_sort[0]]
        # print(len(Ta))
        min_ent, min_t = 0x3f3f3f3f, a_sort[0]
        for t in Ta: # bi-partition
            prop_Dv = len(D[D[a] < t]) / len(D)
            prop_Dv = np.array([prop_Dv,1-prop_Dv])
            prob_Dv_smaller = D[D[a] < t]["salary"].value_counts(normalize=True).get(" >50K",0)
            prob_Dv_bigger = D[D[a] >= t]["salary"].value_counts(normalize=True).get(" >50K",0)
            prob_Dv = np.array([[prob_Dv_smaller,1-prob_Dv_smaller],[prob_Dv_bigger,1-prob_Dv_bigger]])
            prob_Dv = prob_Dv[(prob_Dv[:,0] != 0) & (prob_Dv[:,1] != 0)]
            if len(prob_Dv) == 0:
                sumup = 0
            else:
                sumup = np.sum(prop_Dv * entropy(prob_Dv))
            if min_ent > sumup:
                min_ent = sumup
                min_t = t
        return (entropy(pk) - min_ent, min_t)

cnt = 0
class Node:

    def __init__(self):
        self.branch = {}

    def setLeaf(self,catagory):
        global cnt
        cnt += 1
        if cnt % 1 == 0:
            print("{} - Create leaf: {}".format(cnt,catagory),flush=True)
        self.label = "Leaf"
        self.catagory = catagory
        
    def setBranch(self,attr,value,node,branch_value=None):
        # print("Create branch: {} ({})".format(attr,value),flush=True)
        self.label = "Branch"
        self.attr = attr
        self.branch[value] = node
        if branch_value != None:
            self.branch_value = branch_value

In [56]:
import time

class ID3:

    def __init__(self,train_set=None,validation_set=None,test_set=None,attr_dict=None):
        self.train_set = train_set
        self.validation_set = validation_set
        self.test_set = test_set
        self.attr_dict = attr_dict

    def TreeGenerate(self,dataset,attributes):
        catagory = dataset["salary"].unique()
        node = Node()
        if len(catagory) == 1:
            node.setLeaf(catagory[0])
            return node
        if len(attributes) == 0 or np.sum([len(dataset[a].unique()) for a in attributes]) == len(attributes):
            node.setLeaf(dataset["salary"].value_counts().argmax())
            return node
        max_gain = (-0x3f3f3f3f,None)
        for a in attributes:
            gain = information_gain(dataset,a,self.attr_dict[a])
            if gain[0] > max_gain[0]:
                a_best, max_gain = a, gain
        if self.attr_dict[a_best]: # discrete
            for av in self.train_set[a_best].unique(): # be careful, not dataset!
                Dv = dataset[dataset[a_best] == av]
                if len(Dv) == 0:
                    node.setLeaf(dataset["salary"].value_counts().argmax())
                    return node
                node.setBranch(a_best,av,self.TreeGenerate(Dv,attributes[attributes != a_best]))
        else:
            for flag in ["Smaller","Bigger"]:
                Dv = dataset[dataset[a_best] < max_gain[1]] if flag == "Smaller" else dataset[dataset[a_best] >= max_gain[1]]
                if len(Dv) == 0:
                    node.setLeaf(dataset["salary"].value_counts().argmax())
                    return node
                node.setBranch(a_best,flag,self.TreeGenerate(Dv,attributes),branch_value=max_gain[1])
        return node

    def train(self,train_set=None):
        if train_set != None:
            self.train_set = train_set
        start_time = time.time()
        self.root = self.TreeGenerate(self.train_set,self.train_set.columns.values[:-1])
        print("Time: {:.2f}s".format(time.time()-start_time))

    def validation(self,validation_set=None):
        if validation_set != None:
            self.validation_set = validation_set
        acc = 0
        for i,row in self.validation_set.iterrows():
            p = self.root
            while p.label != "Leaf":
                if self.attr_dict[p.attr]: # discrete
                    p = p.branch[row[p.attr]]
                else:
                    if row[p.attr] < p.branch_value:
                        p = p.branch["Smaller"]
                    else:
                        p = p.branch["Bigger"]
            if p.catagory == row["salary"][:-1]: # be careful of "."
                acc += 1
        acc /= len(self.validation_set)
        return acc

    def test(self,test_set=None):
        if test_set != None:
            self.test_set = test_set
        acc = 0
        for i,row in self.test_set.iterrows():
            p = self.root
            while p.label != "Leaf":
                if self.attr_dict[p.attr]: # discrete
                    p = p.branch[row[p.attr]]
                else:
                    if row[p.attr] < p.branch_value:
                        p = p.branch["Smaller"]
                    else:
                        p = p.branch["Bigger"]
            if p.catagory == row["salary"][:-1]: # be careful of "."
                acc += 1
        acc /= len(self.test_set)
        print("Accurary: {:.2f}%".format(acc * 100))
        return acc

In [57]:
dt = ID3(train_set=train_data,validation_set=validation_data,test_set=test_data,attr_dict=attr_dict)
dt.train()
dt.test()

te leaf:  >50K
2839 - Create leaf:  <=50K
2840 - Create leaf:  >50K
2841 - Create leaf:  <=50K
2842 - Create leaf:  <=50K
2843 - Create leaf:  >50K
2844 - Create leaf:  >50K
2845 - Create leaf:  <=50K
2846 - Create leaf:  >50K
2847 - Create leaf:  <=50K
2848 - Create leaf:  >50K
2849 - Create leaf:  <=50K
2850 - Create leaf:  <=50K
2851 - Create leaf:  >50K
2852 - Create leaf:  <=50K
2853 - Create leaf:  <=50K
2854 - Create leaf:  >50K
2855 - Create leaf:  >50K
2856 - Create leaf:  >50K
2857 - Create leaf:  <=50K
2858 - Create leaf:  >50K
2859 - Create leaf:  >50K
2860 - Create leaf:  >50K
2861 - Create leaf:  <=50K
2862 - Create leaf:  <=50K
2863 - Create leaf:  <=50K
2864 - Create leaf:  >50K
2865 - Create leaf:  >50K
2866 - Create leaf:  <=50K
2867 - Create leaf:  >50K
2868 - Create leaf:  <=50K
2869 - Create leaf:  >50K
2870 - Create leaf:  <=50K
2871 - Create leaf:  >50K
2872 - Create leaf:  >50K
2873 - Create leaf:  <=50K
2874 - Create leaf:  <=50K
2875 - Create leaf:  <=50K
2876

TypeError: unsupported operand type(s) for -: 'float' and 'builtin_function_or_method'

In [58]:
dt.test()

Accurary: 84.15%


0.8414716540753026